<a href="https://colab.research.google.com/github/dolmani38/OCR/blob/main/gate26_0806.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Gate26 pattern dataset

In [1]:
if True:
    from google.colab import drive
    drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import calendar

In [3]:

def addDate(dd,y,m,d,delimiter=''):
    if dd == "ymd":
        data['text'].append(f"{y}{delimiter}{m:02d}{delimiter}{d:02d}")
        data['category'].append('date')
        data['text'].append(f"{y-2000}{delimiter}{m:02d}{delimiter}{d:02d}")
        data['category'].append('date')
        data['text'].append(f"{y}{delimiter}{calendar.month_abbr[m].upper()}{delimiter}{d:02d}")
        data['category'].append('date')
        data['text'].append(f"{y-2000}{delimiter}{calendar.month_abbr[m].upper()}{delimiter}{d:02d}")
        data['category'].append('date')
    elif dd == "dmy":
        data['text'].append(f"{d:02d}{delimiter}{m:02d}{delimiter}{y}")
        data['category'].append('date')
        data['text'].append(f"{d:02d}{delimiter}{m:02d}{delimiter}{y-2000}")
        data['category'].append('date')
        data['text'].append(f"{d:02d}{delimiter}{calendar.month_abbr[m].upper()}{delimiter}{y}")
        data['category'].append('date')
        data['text'].append(f"{d:02d}{delimiter}{calendar.month_abbr[m].upper()}{delimiter}{y-2000}")
        data['category'].append('date')
    elif dd == "mdy":
        data['text'].append(f"{m:02d}{delimiter}{d:02d}{delimiter}{y}")
        data['category'].append('date')
        data['text'].append(f"{m:02d}{delimiter}{d:02d}{delimiter}{y-2000}")
        data['category'].append('date')
        data['text'].append(f"{calendar.month_abbr[m].upper()}{delimiter}{d:02d}{delimiter}{y}")
        data['category'].append('date')
        data['text'].append(f"{calendar.month_abbr[m].upper()}{delimiter}{d:02d}{delimiter}{y-2000}")
        data['category'].append('date')
    else:
        print("Nan")

def addFlightNo(air):
    for i in range(0,999):
        data['text'].append(f"{air}{i:03d}")
        data['category'].append('flight')

def addAirport(city,port):
    data['text'].append(f"{port}")
    data['category'].append('port')
    data['text'].append(f"{city}")
    data['category'].append('city')

def addName(name):
    data['text'].append(f"{name}")
    data['category'].append('name')

def addETC(etc):
    data['text'].append(f"{etc}")
    data['category'].append('etc')

In [4]:
data = {}
data['text'] = []
data['category'] = []

for y in range(2020,2050):
    for m in range(1,13):
        for d in range(1,32):
            addDate("ymd", y, m, d, delimiter='')
            addDate("ymd", y, m, d, delimiter=':')
            addDate("ymd", y, m, d, delimiter='.')
            addDate("ymd", y, m, d, delimiter='-')

            addDate("dmy", y, m, d, delimiter='')
            addDate("dmy", y, m, d, delimiter=':')
            addDate("dmy", y, m, d, delimiter='.')
            addDate("dmy", y, m, d, delimiter='-')

            addDate("mdy", y, m, d, delimiter='')
            addDate("mdy", y, m, d, delimiter=':')
            addDate("mdy", y, m, d, delimiter='.')
            addDate("mdy", y, m, d, delimiter='-')

airline_code = pd.read_csv('https://raw.githubusercontent.com/beanumber/airlines/master/data-raw/airlines.csv')
for cd in airline_code['Code'].values:
    addFlightNo(cd)

airport_code = pd.read_csv('https://raw.githubusercontent.com/datasets/airport-codes/master/data/airport-codes.csv')
for city,cd in airport_code[['municipality','iata_code']].values:
    if pd.isna(cd):
        pass
    else:
        addAirport(str(city).upper(),cd)

female_name = pd.read_csv('https://raw.githubusercontent.com/solvenium/names-dataset/master/dataset/Female_given_names.txt')
for nm in female_name.values[:,0]:
    if pd.isna(nm):
        pass
    else:
        addName(str(nm).upper())

import urllib  # the lib that handles the url stuff

response = urllib.request.urlopen('https://raw.githubusercontent.com/solvenium/names-dataset/master/dataset/Male_given_names.txt')
lines = response.readlines()
for nm in lines: # files are iterable
    addName(str(nm.decode('utf-8').strip()).upper())

response = urllib.request.urlopen('https://raw.githubusercontent.com/dwyl/english-words/master/words.txt')
lines = response.readlines()
for etc in lines: # files are iterable
    addETC(str(etc.decode('utf-8').strip()).upper())

for num in range(0,99999):
    addETC(str(f'{num:05d}'))    

In [5]:
raw_dataset = pd.DataFrame(data)

In [6]:
raw_dataset.to_csv('dataset.csv', encoding='utf-8', index=False)

In [7]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
raw_dataset['label'] = le.fit_transform(raw_dataset['category'])

In [17]:
raw_dataset['label'].unique()

array([1, 3, 5, 0, 4, 2])

In [8]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [10]:
import os
import torch
from transformers import AlbertTokenizer, AlbertForSequenceClassification
from transformers import BertTokenizer, AutoTokenizer, BertForSequenceClassification, AdamW, BertConfig, get_linear_schedule_with_warmup
from sklearn.metrics import classification_report
import time
import random
import datetime
import numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset, random_split

In [11]:
for row in raw_dataset.head().iterrows():
    print(row)

(0, text        20200101
category        date
label              1
Name: 0, dtype: object)
(1, text        200101
category      date
label            1
Name: 1, dtype: object)
(2, text        2020JAN01
category         date
label               1
Name: 2, dtype: object)
(3, text        20JAN01
category       date
label             1
Name: 3, dtype: object)
(4, text        2020:01:01
category          date
label                1
Name: 4, dtype: object)


In [12]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []
labels = []
# For every sentence...
for row in raw_dataset.iterrows():
    # text
    text = str(row[1][0])
    # label
    lb = row[1][2]+0
    labels.append(lb)
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        text,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 64,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                        truncation = True,
                )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Print sentence 0, now as a list of IDs.
print("Tokens (str)      : {}".format([tokenizer.convert_ids_to_tokens(s) for s in input_ids[0].tolist()]))
print('Token IDs:', input_ids[0])

# Training & Validation Split
# Divide up our training set to use 90% for training and 10% for validation.

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, labels)

# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))



/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2329: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Tokens (str)      : ['[CLS]', '2020', '##01', '##01', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
Token IDs: tensor([  101, 12795, 24400, 24400,   102,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
     

In [23]:
# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 64

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [21]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=6)


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [14]:
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [24]:
# Function to calculate the accuracy of our predictions vs labels
def __flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def __format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


def train(epochs=1,train_dataloader=None,validation_dataloader=None,output_dir='/content/drive/MyDrive/Gate26'):

        # Tell pytorch to run this model on the GPU.
        model.cuda()

        # Get all of the model's parameters as a list of tuples.
        params = list(model.named_parameters())

        print('The Albert model has {:} different named parameters.\n'.format(len(params)))
        
        print(model)

        optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

        total_steps = len(train_dataloader) * epochs

        # Create the learning rate scheduler.
        scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                    num_warmup_steps = 0, # Default value in run_glue.py
                                                    num_training_steps = total_steps)
            
        # This training code is based on the `run_glue.py` script here:
        # https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

        # Set the seed value all over the place to make this reproducible.
        seed_val = 42

        random.seed(seed_val)
        np.random.seed(seed_val)
        torch.manual_seed(seed_val)
        torch.cuda.manual_seed_all(seed_val)

        # We'll store a number of quantities such as training and validation loss, 
        # validation accuracy, and timings.
        training_stats = []

        # Measure the total training time for the whole run.
        total_t0 = time.time()

        # For each epoch...
        for epoch_i in range(0, epochs):
            
            # ========================================
            #               Training
            # ========================================
            
            # Perform one full pass over the training set.

            print("")
            print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
            print('Training...')

            # Measure how long the training epoch takes.
            t0 = time.time()

            # Reset the total loss for this epoch.
            total_train_loss = 0

            # Put the model into training mode. Don't be mislead--the call to 
            # `train` just changes the *mode*, it doesn't *perform* the training.
            # `dropout` and `batchnorm` layers behave differently during training
            # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
            model.train()

            # For each batch of training data...
            for step, batch in enumerate(train_dataloader):

                # Progress update every 40 batches.
                if step % 40 == 0 and not step == 0:
                    # Calculate elapsed time in minutes.
                    elapsed = __format_time(time.time() - t0)
                    
                    # Report progress.
                    print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

                # Unpack this training batch from our dataloader. 
                #
                # As we unpack the batch, we'll also copy each tensor to the GPU using the 
                # `to` method.
                #
                # `batch` contains three pytorch tensors:
                #   [0]: input ids 
                #   [1]: attention masks
                #   [2]: labels 
                b_input_ids = batch[0].to(device)
                b_input_mask = batch[1].to(device)
                b_labels = batch[2].to(device)

                #print(b_input_ids)
                #print(b_input_mask)

                # Always clear any previously calculated gradients before performing a
                # backward pass. PyTorch doesn't do this automatically because 
                # accumulating the gradients is "convenient while training RNNs". 
                # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
                model.zero_grad()        

                # Perform a forward pass (evaluate the model on this training batch).
                # The documentation for this `model` function is here: 
                # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
                # It returns different numbers of parameters depending on what arguments
                # arge given and what flags are set. For our useage here, it returns
                # the loss (because we provided labels) and the "logits"--the model
                # outputs prior to activation.

                outputs = model(b_input_ids, 
                                token_type_ids=None, 
                                attention_mask=b_input_mask, 
                                labels=b_labels)

                # Accumulate the training loss over all of the batches so that we can
                # calculate the average loss at the end. `loss` is a Tensor containing a
                # single value; the `.item()` function just returns the Python value 
                # from the tensor.
                loss = outputs[0]
                logits = outputs[1]
                
                total_train_loss += loss.item()

                # Perform a backward pass to calculate the gradients.
                loss.backward()

                # Clip the norm of the gradients to 1.0.
                # This is to help prevent the "exploding gradients" problem.
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

                # Update parameters and take a step using the computed gradient.
                # The optimizer dictates the "update rule"--how the parameters are
                # modified based on their gradients, the learning rate, etc.
                optimizer.step()

                # Update the learning rate.
                scheduler.step()

            # Calculate the average loss over all of the batches.
            avg_train_loss = total_train_loss / len(train_dataloader)            
            
            # Measure how long this epoch took.
            training_time = __format_time(time.time() - t0)

            print("")
            print("  Average training loss: {0:.2f}".format(avg_train_loss))
            print("  Training epcoh took: {:}".format(training_time))
                
            true_labels = []
            pred_labels = []
            # ========================================
            #               Validation
            # ========================================
            # After the completion of each training epoch, measure our performance on
            # our validation set.

            print("")
            print("Running Validation...")

            t0 = time.time()

            # Put the model in evaluation mode--the dropout layers behave differently
            # during evaluation.
            model.eval()

            # Tracking variables 
            total_eval_accuracy = 0
            total_eval_loss = 0
            nb_eval_steps = 0

            # Evaluate data for one epoch
            for batch in validation_dataloader:
                
                # Unpack this training batch from our dataloader. 
                #
                # As we unpack the batch, we'll also copy each tensor to the GPU using 
                # the `to` method.
                #
                # `batch` contains three pytorch tensors:
                #   [0]: input ids 
                #   [1]: attention masks
                #   [2]: labels 
                b_input_ids = batch[0].to(device)
                b_input_mask = batch[1].to(device)
                b_labels = batch[2].to(device)
                for l in batch[2]:
                    true_labels.append(l.item())        
                # Tell pytorch not to bother with constructing the compute graph during
                # the forward pass, since this is only needed for backprop (training).
                with torch.no_grad():        

                    # Forward pass, calculate logit predictions.
                    # token_type_ids is the same as the "segment ids", which 
                    # differentiates sentence 1 and 2 in 2-sentence tasks.
                    # The documentation for this `model` function is here: 
                    # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
                    # Get the "logits" output by the model. The "logits" are the output
                    # values prior to applying an activation function like the softmax.
                    outputs = model(b_input_ids, 
                                    token_type_ids=None, 
                                    attention_mask=b_input_mask,
                                    labels=b_labels)
                loss = outputs[0]
                logits = outputs[1]            
                # Accumulate the validation loss.
                total_eval_loss += loss.item()

                # Move logits and labels to CPU
                logits = logits.detach().cpu().numpy()
                label_ids = b_labels.to('cpu').numpy()
                for li in logits:
                    pred_labels.append(np.argmax(li))
                # Calculate the accuracy for this batch of test sentences, and
                # accumulate it over all batches.
                total_eval_accuracy += __flat_accuracy(logits, label_ids)
                
            print("  classification_report   ")    
            print(classification_report(true_labels,pred_labels))
            # Report the final accuracy for this validation run.
            avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
            print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

            # Calculate the average loss over all of the batches.
            avg_val_loss = total_eval_loss / len(validation_dataloader)
            
            # Measure how long the validation run took.
            validation_time = format_time(time.time() - t0)
            
            print("  Validation Loss: {0:.2f}".format(avg_val_loss))
            print("  Validation took: {:}".format(validation_time))

            # Record all statistics from this epoch.
            training_stats.append(
                {
                    'epoch': epoch_i + 1,
                    'Training Loss': avg_train_loss,
                    'Valid. Loss': avg_val_loss,
                    'Valid. Accur.': avg_val_accuracy,
                    'Training Time': training_time,
                    'Validation Time': validation_time
                }
            )

        # Create output directory if needed
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        print("Saving model to %s" % output_dir)

        # Save a trained model, configuration and tokenizer using `save_pretrained()`.
        # They can then be reloaded using `from_pretrained()`
        model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
        model_to_save.save_pretrained(output_dir)
        tokenizer.save_pretrained(output_dir)

        print("")
        print("Training complete!")
        print("Total training took {:} (h:mm:ss)".format(__format_time(time.time()-total_t0)))

In [ ]:
train(epochs=10, train_dataloader=train_dataloader, validation_dataloader=validation_dataloader)

The Albert model has 201 different named parameters.

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         